In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import argparse
from imutils import paths


# **Moment invariant**

Reading image file

In [3]:
img1 = cv2.imread("Segmented_Teeth/segmented tooth1_1.png")
img2 = cv2.imread("Segmented_Teeth/segmented tooth1_2.png")
img3 = cv2.imread("Segmented_Teeth/segmented_tooth1_3.png")

In [4]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

In [5]:
moments1 = cv2.HuMoments(cv2.moments(img1)).flatten()
print("ORIGINAL MOMENTS for img 1: {}".format(moments1))

moments2 = cv2.HuMoments(cv2.moments(img2)).flatten()
print("ORIGINAL MOMENTS for img 2: {}".format(moments2))

moments3 = cv2.HuMoments(cv2.moments(img3)).flatten()
print("ORIGINAL MOMENTS for img 3: {}".format(moments3))

ORIGINAL MOMENTS for img 1: [9.31673872e-04 3.94367501e-07 1.16589393e-10 1.50416801e-11
 6.29560476e-22 9.44115823e-15 2.07860839e-23]
ORIGINAL MOMENTS for img 2: [9.32487012e-04 4.19574583e-07 7.85269033e-11 1.14095140e-11
 3.37913841e-22 7.05015323e-15 4.94653700e-23]
ORIGINAL MOMENTS for img 3: [1.18052474e-03 9.45728897e-07 1.96083982e-10 8.24694512e-11
 1.04041240e-20 7.77455385e-14 1.31763320e-21]


In [6]:
# for i in range(0,7):
#    moments1[i] = -np.sign(1.0, moments1[i])*np.math.log10(np.abs(moments1[i]))
   
#-1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))

lmoment1 = np.abs(np.log10(moments1))
print('log transformation for moments1',lmoment1)
lmoment2 = np.abs(np.log10(moments2))
print('log transformation for moments2', lmoment2)
lmoment3 = np.abs(np.log10(moments3))
print('log transformation for moments3', lmoment3)

log transformation for moments1 [ 3.03073608  6.40409888  9.93334096 10.82270365 21.20096254 14.02497472
 22.68222732]
log transformation for moments2 [ 3.03035721  6.37719083 10.10498153 10.94273285 21.47119402 14.15180144
 22.30569874]
log transformation for moments3 [ 2.92792491  6.02423334  9.70755788 10.0837069  19.98279448 13.10932452
 20.88020547]


In [7]:
dist = np.linalg.norm(lmoment1 - lmoment2)
print('Distance between moment1 and moment2',dist)

Distance between moment1 and moment2 0.5248576217835638


In [8]:
dist = np.linalg.norm(lmoment1 - lmoment3)
print('Distance between moment1 and moment3',dist)

Distance between moment1 and moment3 2.5142779577407346


# *Data Augmentation*

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import argparse
from imutils import paths
import cv2


In [10]:
datapath="training dataset/crocodiles"
out="training dataset/crocodiles_aug"

datapath1="training dataset/sharks"
out1="training dataset/sharks_aug"

In [11]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(datapath))
imagePaths1 = list(paths.list_images(datapath1))
data=[]
data1=[]

[INFO] loading images...


In [12]:
for imagePath in imagePaths:
    print(imagePath)
    image = cv2.imread(imagePath)
	#cv2.imshow(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    data.append(image)
datanp=np.array(data)

for imagePath1 in imagePaths1:
    print(imagePath1)
    image1 = cv2.imread(imagePath1)
	#cv2.imshow(image)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image1 = cv2.resize(image1, (224, 224))
    image1 = img_to_array(image1)
    image1 = np.expand_dims(image1, axis=0)
    data1.append(image1)
datanp1=np.array(data1)

In [13]:
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0
imageGen=[]

aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0
imageGen1=[]

In [14]:
print("[INFO] generating images...")

# augmentedImages=[]
for i in range(len(datanp)):
        print("Generating is running")
  
        imageGen= aug.flow(datanp[i], batch_size=1, save_to_dir=out,
        save_prefix="image", save_format="jpg")
        for image in imageGen:
	# increment our counter
	        total += 1
	# if we have reached the specified number of examples, break
	# from the loop
	        if (total%6==0):
				#total = 0
		        break

# augmentedImages=[]
for i in range(len(datanp1)):
        print("Generating is running")
  
        imageGen1= aug.flow(datanp1[i], batch_size=1, save_to_dir=out1,
        save_prefix="image", save_format="jpg")
        for image1 in imageGen1:
	# increment our counter
	        total += 1
	# if we have reached the specified number of examples, break
	# from the loop
	        if (total%6==0):
				#total = 0
		        break

[INFO] generating images...


# **SVM classification**

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
dir_path = r'/content/drive/Shareddrives/Graduation Project/Dataset/Teeth images'
categories = ['Afradapis', 'Elephant', 'Hyrax' , 'Masrasector' , 'Sharks']
IMG_SIZE=100
for category in categories:
  path = os.path.join(dir_path,category)
  for img in os.listdir(path):
      img_array=cv2.imread(os.path.join(path,img))
      plt.imshow(img_array)
      plt.show()
      break
  break


In [ ]:
training_data=[]
def create_training_data():
    for category in categories:
        path=os.path.join(dir_path, category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()  

In [ ]:
print(len(training_data))

In [ ]:
lenofimage = len(training_data)

In [ ]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [ ]:
X.shape

In [ ]:
X = X/255.0
X[1]

In [ ]:
y=np.array(y)
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
svc = svm.SVC(kernel='rbf', C=1,gamma='auto').fit(X, y) #rbf model kernal not linear 
svc.fit(X_train, y_train)

# *CNN classification*

In [15]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(2,activation = "softmax")   #Adding the Output Layer
])

In [17]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [18]:
bs=30         #Setting batch size
train_dir = "training dataset"   #Setting training directory
validation_dir = "validation dataset"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

Found 347 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [19]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    # steps_per_epoch=150 // bs,
                    epochs=10,
                    # validation_steps=50 // bs,
                    verbose=2)

Epoch 1/10
12/12 - 8s - loss: 0.6903 - acc: 0.5562 - val_loss: 0.7398 - val_acc: 0.5000 - 8s/epoch - 707ms/step
Epoch 2/10
12/12 - 7s - loss: 0.6173 - acc: 0.6427 - val_loss: 0.6470 - val_acc: 0.5000 - 7s/epoch - 570ms/step
Epoch 3/10
12/12 - 7s - loss: 0.5778 - acc: 0.6455 - val_loss: 0.6955 - val_acc: 0.5000 - 7s/epoch - 572ms/step
Epoch 4/10
12/12 - 8s - loss: 0.6035 - acc: 0.6138 - val_loss: 0.6859 - val_acc: 0.5000 - 8s/epoch - 633ms/step
Epoch 5/10
12/12 - 7s - loss: 0.5858 - acc: 0.6282 - val_loss: 0.6505 - val_acc: 0.5000 - 7s/epoch - 601ms/step
Epoch 6/10
12/12 - 7s - loss: 0.5530 - acc: 0.6427 - val_loss: 0.6138 - val_acc: 0.5000 - 7s/epoch - 599ms/step
Epoch 7/10
12/12 - 8s - loss: 0.5027 - acc: 0.6916 - val_loss: 0.5115 - val_acc: 0.7857 - 8s/epoch - 632ms/step
Epoch 8/10
12/12 - 7s - loss: 0.4818 - acc: 0.8213 - val_loss: 0.3063 - val_acc: 0.7857 - 7s/epoch - 584ms/step
Epoch 9/10
12/12 - 7s - loss: 0.4094 - acc: 0.7954 - val_loss: 0.4432 - val_acc: 0.7143 - 7s/epoch - 586